In [1]:
import requests
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")


2024-05-19 15:06:59.064150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 15:06:59.064256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 15:06:59.197016: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
import os
import shutil

# List of base directories
base_directories = [
    '/kaggle/input/segmented-bob-ross-images',
    '/kaggle/input/paintings-0',
    '/kaggle/input/d',
    '/kaggle/input/edvard-munch-paintings',
    '/kaggle/input/nativity-paintings-dataset',
    '/kaggle/input/abstract-paintings-dataset',
    '/kaggle/input/impressionistlandscapespaintings',
    '/kaggle/input/art-images-drawings-painting-sculpture-engraving',
    '/kaggle/input/oil-painting-images',
    '/kaggle/input/cyclegan-oilpainting-dataset'
]

# Target directory where all images will be moved
target_directory = '/kaggle/working/collected_images'
if not os.path.exists(target_directory):
    os.makedirs(target_directory)

def move_images_to_target(base_dir, target_dir):
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                src_path = os.path.join(root, file)
                dest_path = os.path.join(target_dir, file)
                if os.path.exists(dest_path):
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_dest_path = os.path.join(target_dir, f"{base}_{counter}{extension}")
                    while os.path.exists(new_dest_path):
                        counter += 1
                        new_dest_path = os.path.join(target_dir, f"{base}_{counter}{extension}")
                    dest_path = new_dest_path
                
                shutil.copy2(src_path, dest_path)

for base_directory in base_directories:
    print(base_directory)
    move_images_to_target(base_directory, target_directory)

print("Completed moving all images.")


/kaggle/input/segmented-bob-ross-images
/kaggle/input/paintings-0
/kaggle/input/d
/kaggle/input/edvard-munch-paintings
/kaggle/input/nativity-paintings-dataset
/kaggle/input/abstract-paintings-dataset
/kaggle/input/impressionistlandscapespaintings
/kaggle/input/art-images-drawings-painting-sculpture-engraving
/kaggle/input/oil-painting-images
/kaggle/input/cyclegan-oilpainting-dataset
Completed moving all images.


In [3]:
import random
image_files = [name for name in os.listdir(target_directory) if name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))]
total_images = len(image_files)
num_images_to_remove = total_images // 10
images_to_remove = random.sample(image_files, num_images_to_remove)
for image in images_to_remove:
    os.remove(os.path.join(target_directory, image))
updated_total_images = len([name for name in os.listdir(target_directory) if name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff'))])
print(f'Total number of images in {target_directory} after removal: {updated_total_images}')


Total number of images in /kaggle/working/collected_images after removal: 112462


In [5]:
import csv

In [ ]:

folder_path = '/kaggle/working/collected_images'

# List all image files in the folder
all_images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png'))]

# Batch size
batch_size = 100

# Process images in batches
for i in range(0, len(all_images), batch_size):
    print(i)
    batch_images = all_images[i:i + batch_size]
    image_data = []

    # Process each image in the batch
    for filename in batch_images:
        img_path = os.path.join(folder_path, filename)
        
        try:
            raw_image = Image.open(img_path).convert('RGB')
        except UnidentifiedImageError:
            print(f'Skipping file {filename} as it is not a valid image.')
            continue

        # Conditional image captioning
        text = "a photography of"
        inputs = processor(raw_image, text, return_tensors="pt")

        # Unconditional image captioning
        inputs = processor(raw_image, return_tensors="pt")

        out = model.generate(**inputs)
        description = processor.decode(out[0], skip_special_tokens=True)
        

        # Append the image ID and description to the list
        image_data.append({"image_id": filename, "description": description})

    # Specify the CSV file path for the current batch
    csv_file_path = f'/kaggle/working/image_descriptions_batch_{i//batch_size + 1}.csv'

    # Write the image data to the CSV file
    with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=["image_id", "description"])
        writer.writeheader()
        writer.writerows(image_data)

    print(f'Descriptions for batch {i//batch_size + 1} saved to {csv_file_path}')

print("Completed processing all images.")

0
Descriptions for batch 1 saved to /kaggle/working/image_descriptions_batch_1.csv
100
